In [1]:
import requests
import re
from bs4 import BeautifulSoup
import pandas as pd
import time
import numpy as np

# Страница с которой берем квартиры

In [2]:
district = 'http://www.cian.ru/cat.php?deal_type=sale&district%5B0%5D=13&district%5B1%5D=14&district%5B2%5D=15&\
district%5B3%5D=16&district%5B4%5D=17&district%5B5%5D=18&district%5B6%5D=19&district%5B7%5D=20&district%5B8%5D=21\
&district%5B9%5D=22&engine_version=2&offer_type=flat&p={}&room1=1&room2=1&room3=1&room4=1&room5=1&room6=1'

In [3]:
district

'http://www.cian.ru/cat.php?deal_type=sale&district%5B0%5D=13&district%5B1%5D=14&district%5B2%5D=15&district%5B3%5D=16&district%5B4%5D=17&district%5B5%5D=18&district%5B6%5D=19&district%5B7%5D=20&district%5B8%5D=21&district%5B9%5D=22&engine_version=2&offer_type=flat&p={}&room1=1&room2=1&room3=1&room4=1&room5=1&room6=1'

# Собираем базу ссылок на страницы квартир

Для начала посмотрим на первую страницу - по ней определимся со всеми остальными (всего их 30)

In [4]:
first_page = requests.get(district.format(1))
first_page = first_page.text
first_page = BeautifulSoup(first_page, 'lxml')

Все квартиры лежат теге div с id = content
Положим нужный кусок html кода в переменную

In [5]:
flats = first_page.html.body.findAll('div', id='content')

Углубляемся дальше - видим div c классом serp-list

In [6]:
flats = flats[0].findAll('div', attrs = {'class':'serp-list'})

In [7]:
flats = flats[0]

In [8]:
flats_cian = flats.findAll('div',\
              attrs = {'ng-class':"{'serp-item_removed': offer.remove.state, 'serp-item_popup-opened': isPopupOpen}"})

Соберем ссылки на страницы квартир с первой страницы поиска

In [10]:
flats_one_page = []
for i in range(len(flats_cian)):
    flats_one_page.append(flats_cian[i].attrs['href'])

In [11]:
flats_one_page[0:5] # Посмотрим на первые пять

['http://www.cian.ru/sale/flat/150922264/',
 'http://www.cian.ru/sale/flat/2914021/',
 'http://www.cian.ru/sale/flat/47073122/',
 'http://www.cian.ru/sale/flat/149031353/',
 'http://www.cian.ru/sale/flat/149031259/']

Обернем сбор ссылок в функцию

In [12]:
def page_grabber(page):
    flats_links = [] # список с ссылками с одной страницы поиска
    p = len(page)
    for i in range(p):
        flats_links.append(page[i].attrs['href']) # один элемент - ссылка (то есть атрибут href тега div)
    return(flats_links)

Соберем все квартиры:

In [13]:
links = [] # Список, который заполним всеми квартирами (только те, что размещены внутри самомго сайта)

for i in range(1,31):
    # Получаем нужную страницу
    search_page = requests.get(district.format(i))
    search_page = search_page.text
    search_page = BeautifulSoup(search_page, 'lxml')
    
    # Внутри нужной страницы добираемся до таблицы с квартирами
    flats = search_page.html.body.findAll('div', id='content')
    flats = flats[0].findAll('div', attrs = {'class':'serp-list'})
    
    # В этой переменной будет лежать нужный кусок страницы (в каждой итерации новая страница)
    flats_cian = flats[0].findAll('div',\
              attrs = {'ng-class':"{'serp-item_removed': offer.remove.state, 'serp-item_popup-opened': isPopupOpen}"})
    
    # Берем ссылки с текущей страницы и кладем в наш общий список ссылок
    links = links + page_grabber(flats_cian)
    
    # Каждую итерацию проверяем, что все нормально работает 
    print('The page number {} wroks well\nIts requests.ok is {}'.format(i, requests.get(district.format(i)).ok))

The page number 1 wroks well
Its requests.ok is True
The page number 2 wroks well
Its requests.ok is True
The page number 3 wroks well
Its requests.ok is True
The page number 4 wroks well
Its requests.ok is True
The page number 5 wroks well
Its requests.ok is True
The page number 6 wroks well
Its requests.ok is True
The page number 7 wroks well
Its requests.ok is True
The page number 8 wroks well
Its requests.ok is True
The page number 9 wroks well
Its requests.ok is True
The page number 10 wroks well
Its requests.ok is True
The page number 11 wroks well
Its requests.ok is True
The page number 12 wroks well
Its requests.ok is True
The page number 13 wroks well
Its requests.ok is True
The page number 14 wroks well
Its requests.ok is True
The page number 15 wroks well
Its requests.ok is True
The page number 16 wroks well
Its requests.ok is True
The page number 17 wroks well
Its requests.ok is True
The page number 18 wroks well
Its requests.ok is True
The page number 19 wroks well
Its req

In [14]:
len(links)

840

Собрали ссылки на 840 квартир.

---

# Напишем функции, которые берут нужную информации по каждой квартире

## Посмотрим, что нужно собрать
- N – номер квартиры по порядку
- Rooms – число комнат
- Price – цена
- Totsp – общая площадь квартиры, кв. м.
- Livesp – жилая площадь квартиры, кв. м.
- Kitsp – площадь кухни, кв. м.
- Dist – расстояние от центра в км. (можно по Пифагору)
- Metrdist – расстояние до метро в минутах
- Walk – 1 – если пешком от метро, 0 – если на транспорте
- Brick – 1 – кирпичный/монолит/жб, 0 – другой
- Tel – 1 – если есть, 0 – нет
- Bal – 1 – есть балкон/лоджия, 0 – нет
- Floor – номер этажа, на котором расположена квартира
- Nfloors – всего этажей в доме
- New – 1 – первичный рынок, 0 - вторичка

 Посмотрим на вторую страницу (там доступно больше информации, поэтому по ней будет удобно настраивать парсер).

In [283]:
links[1]

'http://www.cian.ru/sale/flat/150695572/'

### Собираем данные

Посмотрев на код страницы видим, что много информации лежит в таблице с классом object_descr_tab. Захватим этот кусок сайта.

############################################################################################################################

In [410]:
flat_page = requests.get(links[1])
flat_page = flat_page.text
flat_page = BeautifulSoup(flat_page, 'lxml')

In [411]:
inf_table = flat_page.findAll('table', class_ = 'object_descr_tab')
inf_table = inf_table[0]

############################################################################################################################

### Пойдем по нужным параметрам подряд

#### Число комнат

In [412]:
room = inf_table.findAll('div', class_='object_descr_title')

In [413]:
room = room[0]

In [414]:
room = re.findall(r'.?-комн. кв.' ,str(room))[0]

In [415]:
# Берем первый символ, поскольку в фильтре на циане задано, что комнат может быть от 1 до 5 (больше - это "многокомн. кв.")
int(room[0])

4

Обернем это дело  функцию

На сайте есть страницы квартир, где вместо количества комнат стоят слова "многокомн. кв." - в этом случае будем считать, что квартир там 6. (как правио это большие квартиры от 200 кв. м., поэтому 6 еомнат - логичное предположение)

In [372]:
def room_grabber(table):
    room = table.findAll('div', class_='object_descr_title')
    try:
        room = re.findall(r'.?-комн. кв.' ,str(room[0]))[0]
    except:
        if re.findall(r'многокомн' ,str(room[0]))[0] == 'многокомн':
            room = [6]
    return(int(room[0]))

In [373]:
room_grabber(inf_table)

6

#### Цена

In [416]:
price = inf_table.findAll('div', class_='object_descr_price_box')

In [417]:
price[0].findAll('div')[0]

<div class="object_descr_price">
            
                

55 000 000 руб.

            
            


            
        </div>

In [418]:
p1 = price[0].findAll('div')[0].string.strip()
print(p1[-5:]) # проверим, что выкинем только " руб."
p1 = int(p1[:-5].replace(' ', '')) # выкинули " руб." и убрали пробелы
print(p1)

 руб.
55000000


In [419]:
p2 = price[0].findAll('div')[1].string.strip()

In [420]:
p2 = p2.replace(',', '.')

In [421]:
p2 = float(p2)
print(p2)

55000000.0


Обернем поиск цены в функцию

In [354]:
def price_grabber(table):
    '''
    Будем возвращать цену списком. Судя по всему ЦИАН предоставил нам скрытый тег, в котором лежит цена в удобном для нас
    виде - на всякий случай будем собирать цену в двух вариантах (можно будет проверить + если один вариант отпал - другой будет доступен).
    В качестве нулевых значений возьмем NaN
    '''
    price = table.findAll('div', class_='object_descr_price_box')
    
    p1 = np.nan
    p2 = np.nan
    
    try:
        p1 = price[0].findAll('div')[0].string.strip()
        p1 = int(p1[:-5].replace(' ', ''))
    except:
        pass
    
    try:
        p2 = price[0].findAll('div')[1].string.strip()
        p2 = float(p2.replace(',', '.'))
    except:
        pass
    
    return([p1,p2])

In [355]:
price_grabber(inf_table)

[nan, 234530831.08]

#### Расстояние до метро в минутах + Walk

По тегу div находим нужный кусок. 

In [30]:
metro = inf_table.findAll('div', class_='object_descr_metro')

In [31]:
metro = metro[0].findAll('span', class_='object_item_metro_comment')[0].string

Через регулярный выражения найдем кусок с числами - это будут минуты

In [32]:
metrdist = int(re.findall(r'\d+', metro)[0])

In [33]:
metrdist

9

Для определения walk сначала выкинем из строки все пробелы и переносы строк, что бы было проще использовать регулярное выражение. Затем найдем все, что будет идти после "мин." - это и есть наш способ передвижения до дома.

In [34]:
walk = metro.replace(' ', '')
walk = walk.replace('\n', '')

In [35]:
walk = re.findall(r'мин\..+', walk)[0][4:]

In [36]:
walk

'пешком'

Снова обернем поиск этих двух величин в функцию

In [335]:
def metro(table):
    try:
        # Нужный кусок html
        metro = table.findAll('div', class_='object_descr_metro')
        metro = metro[0].findAll('span', class_='object_item_metro_comment')[0].string

        # Минуты
        metrdist = int(re.findall(r'\d+', metro)[0])

        # Walk
        walk = metro.replace(' ', '')
        walk = walk.replace('\n', '')
        walk = re.findall(r'мин\..+', walk)[0][4:]
        if walk == 'пешком':
            walk = 1
        elif walk == []:
            walk = np.nan
        else:
            walk = 0
    except:
        metrdist = np.nan
        walk = np.nan
        
    return({'metrdist': metrdist, 'walk': walk})

In [336]:
metro(inf_table)

{'metrdist': nan, 'walk': nan}

#### Материал стен: кирпичный/монолит/жб

Сначала найдем нужный div (данные лежат в таблице под картой). Причем надо помнить, что эта секция может отсутствовать на странице квартиры, так что это надо учесть при поиске.

In [273]:
house_table = flat_page.findAll('div', class_='offer_container object_descr cleared')

In [274]:
house_table = house_table[0].findAll('div', class_='bti__inner')

In [276]:
len(house_table)

1

В таблице, содержащей данные о доме, найдем первый тег tbody, так как именно в нем лежит информация о материале стен

In [245]:
first_table = house_table[0].findAll('tbody')[0]

In [249]:
first_table_tr = first_table.findAll('tr')

Так будем искать нужные куски инфрмации

In [260]:
first_table_tr[1].th

<th>
<div class="bti__data__name">Материалы стен</div>
<div class="bti__data__stretch"></div>
</th>

In [261]:
first_table_tr[1].td

<td>кирпичный</td>

In [280]:
def brick_grabber(flat_page):
    
    # Ищем на странице нужный кусок html кода
    house_table = flat_page.findAll('div', class_='offer_container object_descr cleared')
    house_table = house_table[0].findAll('div', class_='bti__inner')
    
    # Проверка на случай, если искомая секция вобще отсутствует на странице данной квартиры
    if house_table == []:
        brick = np.nan
    else:
        # Углубляемся в нужный кусок html кода
        first_table = house_table[0].findAll('tbody')[0]
        first_table_tr = first_table.findAll('tr') # Собрали все tr, то есть все секции первой малой таблицы

        #Ищем нужный th с информацией о материале
        for i in range(1,len(first_table_tr)):
            try:
                brick_str = re.findall(r'Материалы стен', str(first_table_tr[i].th))[0]
                if brick_str == 'Материалы стен':
                    # Ищем один из нужных нам флажков - кирпич, монолит или варианты ж/б
                    brick = re.findall(r'кирпич|монолит|ж??б|желез.+бетон', str(first_table_tr[i].td))
            except:
                pass

        # Проверяем результат регулярного выражения
        if len(brick) == 0:
            brick = 0
        else:
            brick = 1
        
    return(brick)

In [281]:
brick_grabber(flat_page)

nan

#### Расстояние от центра в км.

Для начала найдем координаты нашего дома. Для этого используем карту. Как видно из кода в куске страницы, посвященной карте, можно выудить ссылку, в которой прописаны искомые координты.

In [284]:
flat_page.find('div', attrs={'class':'map_info_button_extend'}).contents[1]

<a href="http://map.cian.ru/?deal_type=2&amp;flats=yes&amp;center=55.762586%2C37.597809&amp;room0=1&amp;room1=1&amp;room2=1&amp;room3=1&amp;room4=1" rel="nofollow" target="_blank"><i></i>Посмотреть предложения рядом</a>

In [285]:
coords = flat_page.find('div', attrs={'class':'map_info_button_extend'}).contents[1]
coords = re.split('&amp|center=|%2C', str(coords))
coords_list = []
for item in coords:
    if item[0].isdigit():
        coords_list.append(item)
lat = float(coords_list[0])
lon = float(coords_list[1])

In [286]:
lat, lon

(55.762586, 37.597809)

In [288]:
def getCoords(flat_page):
    coords = flat_page.find('div', attrs={'class':'map_info_button_extend'}).contents[1]
    coords = re.split('&amp|center=|%2C', str(coords))
    coords_list = []
    for item in coords:
        if item[0].isdigit():
            coords_list.append(item)
    lat = float(coords_list[0])
    lon = float(coords_list[1])
    return lat, lon

In [296]:
getCoords(flat_page)

(55.762586, 37.597809)

Теперь надо померить расстояние. В качестве центра будем брать "[Нулевой километр](https://tools.wmflabs.org/geohack/geohack.php?language=ru&pagename=%D0%9D%D1%83%D0%BB%D0%B5%D0%B2%D0%BE%D0%B9_%D0%BA%D0%B8%D0%BB%D0%BE%D0%BC%D0%B5%D1%82%D1%80&params=55.755831_N_37.617673_E)". Расстоние будем считать по Пифагору.

In [290]:
null_coords = (55.755831, 37.617673)

Зная расстояния между двумя координатами в градусах, можно вычислить расстояние на земной поверхности и перевести его в километры.

Используем формулы, приведенные ниже, где:
- Фa и Фb — широты данных пунктов
- Лa, Лb — долготы данных пунктов
- d — расстояние между пунктами
- R - раудис Земли

$$d = arccos {sin(Фa)·sin(Фb) + cos(Фa)·cos(Фb)·cos(Лa - Лb)}$$

$$L = d·R$$

In [299]:
# Широта (latitude):
lat1 = getCoords(flat_page)[0]
lat2 = null_coords[0]

# Долгота (longtude):
lon1 = getCoords(flat_page)[1]
lon2 = null_coords[1]

# Радиус Земли
R = 6371

In [300]:
d = np.arccos(
    np.sin(lat1)*np.sin(lat2) + np.cos(lat1)*np.cos(lat2)*np.cos(lat1-lat2)
)
L = d*R
L

52.636699067230666

Сделаем функцию, считающую расстояние

In [306]:
house_coords = getCoords(flat_page)
def dist_grabber(house_coords):
    null_coords = (55.755831, 37.617673)
    
    # Радиус Земли
    R = 6371
    
    # Широта (latitude):
    lat1 = house_coords[0]
    lat2 = null_coords[0]

    # Долгота (longtude):
    lon1 = house_coords[1]
    lon2 = null_coords[1]
    
    d = np.arccos(
    np.sin(lat1)*np.sin(lat2) + np.cos(lat1)*np.cos(lat2)*np.cos(lat1-lat2)
    )
    L = d*R
    return(L)

In [307]:
dist_grabber(house_coords)

52.636699067230666

### Далее нужно зайти в таблицу поглубже

############################################################################################################################

In [158]:
inner_table = inf_table.findAll('table', class_ = 'object_descr_props flat sale')
inner_table = inner_table[0]

Сложим все теги td (по сути все пункты маленькой таблицы) в список

In [159]:
table_td = inner_table.findAll('tr') # Теперь тут лежит внутренняя таблица (список td)

############################################################################################################################

#### Общая площадь квартиры, кв. м. + Площадь кухни, кв. м. + Жилая площадь квартиры, кв. м.

Начнем с общей площади. Найдем кусок с метрами.

In [112]:
totsp = re.findall(r'\d+\,?\d+\xa0м', str(table_td[4].td))
totsp

['117\xa0м']

In [113]:
totsp = totsp[0].replace(',', '.')
totsp

'117\xa0м'

На случай, если число дробное, то заменили запятую на точку и добавили возможность поиска такой комбинации в регулярное выражение

In [114]:
totsp = float(re.findall(r'\d+\.?\d+', totsp)[0])
totsp

117.0

In [115]:
#Ищем нужный th с общей площадью квартиры
for i in range(1,len(table_td)):
    # Положим сюда строку "Общая площадь" и поней будем искать нужный th
    # (на случай, если кол-во параметров может отличаться в разных квартирах)
    try:
        totsp_str = re.findall(r'Общая площадь' ,table_td[i].th.string)[0]
        if totsp_str == 'Общая площадь':
            totsp = re.findall(r'\d+\,?\d+\xa0м', str(table_td[i].td))# Найдем кусок с метрами
            # На случай, если число дробное, то заменили запятую на точку и добавили возможность 
            #поиска такой комбинации в регулярное выражение
            totsp = totsp[0].replace(',', '.')
            totsp = float(re.findall(r'\d+\.?\d+', totsp)[0])
            print(totsp)
    except:
        pass

117.0


Снова завернем работающую функцию

Однако, сделаем небольшую надстройку: если в графе о площади будет отсутствовать информация, то re.findall вернет пустой список. В этом случае заполним значение площади nan для данной квартиры

In [124]:
def totsp_grabber(inner_table):
    
    # Теперь тут лежит внутренняя таблица (список td)
    table_totsp = inner_table.findAll('tr')
    
    #Ищем нужный th с общей площадью квартиры
    for i in range(1,len(table_totsp)):
        # Положим сюда строку "Общая площадь" и поней будем искать нужный th
        # (на случай, если кол-во параметров может отличаться в разных квартирах)
        try:
            totsp_str = re.findall(r'Общая площадь' ,table_totsp[i].th.string)[0]
            if totsp_str == 'Общая площадь':
                totsp = re.findall(r'\d+\,?\d+\xa0м', str(table_totsp[i].td))# Найдем кусок с метрами
                
                # Надстройка на случай, если графа с площадью пуста
                if totsp == []:
                    totsp = np.nan
                    break
                
                # На случай, если число дробное, то заменили запятую на точку и добавили возможность 
                #поиска такой комбинации в регулярное выражение
                totsp = totsp[0].replace(',', '.')
                totsp = float(re.findall(r'\d+\.?\d+', totsp)[0])
        except:
            pass
    return(totsp)

In [125]:
totsp_grabber(inner_table)

117.0

Поскольку площадь кухни и жилая площадь будут искаться аналогично, напишем функцию, которая будет искать все три параметра

In [118]:
def sp_grabber(inner_table, sp_pl = ['Общая площадь', 'Площадь кухни', 'Жилая площадь']):
    
    # Теперь тут лежит внутренняя таблица (список td)
    table_td = inner_table.findAll('tr')
    
    # Список, куда будем класть наши значения искомых площадей
    sp_list = []
    
    # Перебираем три метста
    for s in sp_pl:
        
        # Поиск нужной цифры
        for i in range(1,len(table_td)):
            try:
                sp_str = re.findall(r'{}'.format(s) ,table_td[i].th.string)[0] # В качестве s ставим искмую сейчас площадь
                if sp_str == '{}'.format(s):
                    sp = re.findall(r'\d+\,?\d+\xa0м', str(table_td[i].td))

                    # Надстройка на случай, если графа с площадью пуста
                    if sp == []:
                        sp = np.nan
                        sp_list.append(sp) # Добавим в список значение так же, если нужной информации нет
                        break

                    sp = sp[0].replace(',', '.')
                    sp = float(re.findall(r'\d+\.?\d+', sp)[0])
                    
                    # Добавим в список значение очередной площади
                    sp_list.append(sp)
            except:
                pass
    
    # Словарь, в котором подпишем цифры удобным названием переменных
    sp_dist = {}
    sp_dist['totsp'] = sp_list[0]
    sp_dist['kitsp'] = sp_list[1]
    sp_dist['livesp'] = sp_list[2]
    
    # Вернем словарь со всеми нужными значениями
    return(sp_dist)

In [119]:
sp_grabber(inner_table, sp_pl = ['Общая площадь', 'Площадь кухни', 'Жилая площадь'])

{'kitsp': 15.0, 'livesp': 89.0, 'totsp': 117.0}

#### Номер этажа, на котором расположена квартира + Всего этажей в доме

Вся информация про этажи находится во внутренней таблице, так что фнкция, которая ее вытаскивает похожа на предыдущую. Информация может быть представлена тремя вариантами:
- число
- число / число
- информации нет

Последний случай будет описан аналогично тому, как в предыдущей функции, если число одно, то будет возвращаться список длиной 1, а если чисел - два, то первый элемент списка будет этаж квартиры, второй - число отражающее количество этажей в доме всего.

In [191]:
def floor_grabber(inner_table):
    
    # Теперь тут лежит внутренняя таблица (список td)
    table_td = inner_table.findAll('tr')
    
    # Вытаскиваем этажи
    for i in range(1,len(table_td)):
        try:
            th = re.findall(r'Этаж' ,table_td[i].th.string)[0]
            if th == 'Этаж':
                floor_td = table_td[i].td # тег со всей информацией об этажах
                floor_td = re.findall(r'\d+', str(floor_td)) # Теперь в переенной лежит список
                # Одно число - только этаж квартиры, Два - еще суммарное количество этажей, Пустой - этой информции нет
        except:
            pass
    
    # Определяем какую информацию дать на выход
    if len(floor_td) == 0:
        floor = np.nan
        nfloor = np.nan
    elif len(floor_td) == 1:
        floor = int(floor_td[0])
        nfloor = np.nan
    else:
        floor = int(floor_td[0])
        nfloor = int(floor_td[1])

    return({'floor':floor, 'n_floor':nfloor})

In [192]:
floor_grabber(inner_table)

{'floor': 2, 'n_floor': 4}

#### Балкон

Поиск балкона тоже будет похож на предыдущие функции. Но теперь нам важно найти любую цифру в графе "Балкон" или слово "есть".

In [262]:
def bal_grabber(inner_table):
    
    # Теперь тут лежит внутренняя таблица (список td)
    table_td = inner_table.findAll('tr')
    
    #Ищем нужный th с общей площадью квартиры
    for i in range(1,len(table_td)):
        # Положим сюда строку "Балкон" и поней будем искать нужный th
        # (на случай, если кол-во параметров может отличаться в разных квартирах)
        try:
            bal_str = re.findall(r'Балкон' ,table_td[i].th.string)[0]
            if bal_str == 'Балкон':
                bal = re.findall(r'i>\d+|есть', str(table_td[i].td))# Найдем кусок с метрами
                # Если балконов нет, то реулярное выражение ничего не найдет и вернет пустой список
                if bal == []:
                    bal = 0
                else:
                    bal = 1
        except:
            pass
        
    return(bal)

In [263]:
bal_grabber(inner_table)

0

#### Телефон

И снова функция похожая на предыдущие, так же с небольшими поправками

In [325]:
def tel_grabber(inner_table):
    
    # Теперь тут лежит внутренняя таблица (список td)
    table_td = inner_table.findAll('tr')
    
    # На некоторых страницах отстутствует графа "Телефон", поэтому введем проверку на наличие этой строки с помощью индикатора
    th_count = 0
    
    # Смотрим на td в поисках графы "Телефон"
    for i in range(1,len(table_td)):
        try:
            # Накручиваем значение индикатора
            th_count = th_count + 1
            
            th = re.findall(r'Телефон' ,table_td[i].th.string)[0]
            
            if th == 'Телефон':
                th_count = 0
                
                tel = table_td[i].td.string
                
                # Проверка на текст в нужном куске html кода
                if tel == 'да':
                    tel = 1
                else:
                    tel = 0
        except:
            pass
    
    # Проверка индикатора
    if th_count == len(table_td)-1:
        tel = 0
    
    return(tel)

In [326]:
tel_grabber(inner_table)

0

#### Рынок (первичный / вторичный)

Этот параметр будем искать по корню "вторич" в графе "Тип дома"

In [208]:
def new_grabber(inner_table):
    
    # Теперь тут лежит внутренняя таблица (список td)
    table_td = inner_table.findAll('tr')
    
    # Идем по всем td в поисках графы 'Тип дома'
    for i in range(1,len(table_td)):
        try:
            new_str = re.findall(r'Тип дома' ,table_td[i].th.string)[0]
            if new_str == 'Тип дома':
                new = re.findall(r'вторич', str(table_td[i].td))
                
                # Проверка на наличия корня "вторич" в данной графе таблице
                if new == []:
                    new = 1
                else:
                    new = 0
        except:
            pass
        
    return(new)

In [209]:
new_grabber(inner_table)

0

# Теперь соберем базу квартир

Напишем код, кторый достанет все квартиры

- N – номер квартиры по порядку
- Rooms – число комнат
- Price – цена
- Totsp – общая площадь квартиры, кв. м.
- Livesp – жилая площадь квартиры, кв. м.
- Kitsp – площадь кухни, кв. м.
- Dist – расстояние от центра в км. (можно по Пифагору)
- Metrdist – расстояние до метро в минутах
- Walk – 1 – если пешком от метро, 0 – если на транспорте
- Brick – 1 – кирпичный/монолит/жб, 0 – другой
- Tel – 1 – если есть, 0 – нет
- Bal – 1 – есть балкон/лоджия, 0 – нет
- Floor – номер этажа, на котором расположена квартира
- Nfloors – всего этажей в доме
- New – 1 – первичный рынок, 0 - вторичка

Добавим еще пару параметров

- Price_duplicate - дубликат цены, полученный из соседнего тега (подробное обьяснение в описании функции) - для проверки
- Link - ссылка на данную квартиру

In [374]:
flats_dict = {}
N = 0

for l in links:
    
    # Это будет номер квартиры
    N = N + 1
    
    # Соберем все, что написано выше, вместе
    flat_page = requests.get(l)
    flat_page = flat_page.text
    flat_page = BeautifulSoup(flat_page, 'lxml')
    inf_table = flat_page.findAll('table', class_ = 'object_descr_tab')
    inf_table = inf_table[0]
    inner_table = inf_table.findAll('table', class_ = 'object_descr_props flat sale')
    inner_table = inner_table[0]
    
    getCoords(flat_page)
    house_coords = getCoords(flat_page)
    
    flats_dict[N] = {
        'Rooms': room_grabber(inf_table),
        'Price': price_grabber(inf_table)[0],
        'Price_duplicate': price_grabber(inf_table)[1],
        'Totsp': sp_grabber(inner_table, sp_pl = ['Общая площадь', 'Площадь кухни', 'Жилая площадь'])['totsp'],
        'Livesp': sp_grabber(inner_table, sp_pl = ['Общая площадь', 'Площадь кухни', 'Жилая площадь'])['kitsp'],
        'Kitsp': sp_grabber(inner_table, sp_pl = ['Общая площадь', 'Площадь кухни', 'Жилая площадь'])['livesp'],
        'Dist': dist_grabber(house_coords),
        'Metrdist': metro(inf_table)['metrdist'],
        'Walk': metro(inf_table)['walk'],
        'Brick': brick_grabber(flat_page),
        'Tel': tel_grabber(inner_table),
        'Bal': bal_grabber(inner_table),
        'Floor': floor_grabber(inner_table)['floor'],
        'Nfloors': floor_grabber(inner_table)['n_floor'],
        'New': new_grabber(inner_table),
        'Link': l
    }
    
    print('Page {} is working well'.format(N))

Page 1 is working well
Page 2 is working well
Page 3 is working well
Page 4 is working well
Page 5 is working well
Page 6 is working well
Page 7 is working well
Page 8 is working well
Page 9 is working well
Page 10 is working well
Page 11 is working well
Page 12 is working well
Page 13 is working well
Page 14 is working well
Page 15 is working well
Page 16 is working well
Page 17 is working well
Page 18 is working well
Page 19 is working well
Page 20 is working well
Page 21 is working well
Page 22 is working well
Page 23 is working well
Page 24 is working well
Page 25 is working well
Page 26 is working well
Page 27 is working well
Page 28 is working well
Page 29 is working well
Page 30 is working well
Page 31 is working well
Page 32 is working well
Page 33 is working well
Page 34 is working well
Page 35 is working well
Page 36 is working well
Page 37 is working well
Page 38 is working well
Page 39 is working well
Page 40 is working well
Page 41 is working well
Page 42 is working well
P

Мы собрали данные по всем найденным квартирам. Теперь положим это дело в удобный DataFrame

In [378]:
df_flats = pd.DataFrame(flats_dict).T

Посмотрим на результат парсинга:

In [425]:
df_flats.describe()

,Bal,Brick,Dist,Floor,Kitsp,Link,Livesp,Metrdist,New,Nfloors,Price,Price_duplicate,Rooms,Tel,Totsp,Walk
count,840,482,840.000000,840,750,840,311,824,840,798,818,840,840,840,840,824
unique,2,2,472.000000,23,262,833,82,19,2,32,410,430,6,2,305,2
top,0,1,155.575171,2,30,http://www.cian.ru/sale/flat/147839229/,10,5,0,5,16500000,16500000,2,0,38,1
freq,427,391,67.000000,160,42,2,77,203,659,115,19,19,341,500,20,778


In [401]:
for c in df_flats.columns:
    print(c, len(df_flats[c][df_flats[c].isnull()]))

Bal 0
Brick 358
Dist 0
Floor 0
Kitsp 90
Link 0
Livesp 529
Metrdist 16
New 0
Nfloors 42
Price 22
Price_duplicate 0
Rooms 0
Tel 0
Totsp 0
Walk 16


Сохраним результат в csv файл

In [422]:
df_flats.to_csv('flats_from_CIAN.csv')